### Building an Action Server

Once you have defined the action ([here](115-ROS_action_definition.ipynb)) the next step is to write the python code that will define the server.  We first define the server, importing the message type we defined above and defining a callback that returns the result.  

In [ ]:
os.chdir(workspace)

In [ ]:
%%writefile timer_action_server.py

import time
import rclpy
from rclpy.action import ActionServer
from rclpy.node import Node

from timer.action import Timer

class TimerActionServer(Node):

    def __init__(self):
        super().__init__('timer_action_server')
        self._action_server = ActionServer(
            self,Timer,'timer',self.execute_callback)
        print('set up and waiting')

    def execute_callback(self, goal_handle):
        self.get_logger().info('Executing goal...')
        feedback_msg = Timer.Feedback()
        duration = goal_handle.request.duration

        start = time.time()
        while (time.time()-start) < goal_handle.request.duration:
            feedback_msg.sec_left = duration - int(time.time()-start)
            self.get_logger().info('Feedback: {0}'.format(feedback_msg.sec_left))
            goal_handle.publish_feedback(feedback_msg)
            time.sleep(1)

        goal_handle.succeed()

        result = Timer.Result()
        result.done = 0
        return result

def main(args=None):
    rclpy.init(args=args)

    timer_action_server = TimerActionServer()

    rclpy.spin(timer_action_server)


if __name__ == '__main__':
    main()